In [1]:
#Notebook for experiments in gathering data

In [2]:
import re
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [24]:
hwr_director_df = pd.read_csv('raw_data/hwreporter/hollywoodreporter_director_urls_records.csv')
hwr_films_df = pd.read_csv('raw_data/hwreporter/hollywoodreporter_film_urls_records.csv')

In [25]:
hwr_films_df = hwr_films_df.drop('uid', axis=1)
hwr_director_df = hwr_director_df.drop('uid', axis=1)

In [27]:
len(hwr_films_df), len(hwr_director_df)

(90154, 63999)

In [52]:
hwr_df = pd.concat([hwr_films_df, hwr_director_df])
print("Before drop:", len(hwr_df))

hwr_df.drop_duplicates(keep='first',inplace=True) 
print("After drop:", len(hwr_df))

hwr_df.insert(0, "media", "hollywood_reporter")
hwr_df['url'] = 'https://www.hollywoodreporter.com' + hwr_df['url']
hwr_df.head()

Before drop: 154153
After drop: 101918


,media,category,url
0,hollywood_reporter,news,https://www.hollywoodreporter.com/news/golden-...
1,hollywood_reporter,news,https://www.hollywoodreporter.com/news/critics...
2,hollywood_reporter,news,https://www.hollywoodreporter.com/news/wandas-...
3,hollywood_reporter,news,https://www.hollywoodreporter.com/news/oscars-...
4,hollywood_reporter,news,https://www.hollywoodreporter.com/news/sony-ex...


In [74]:
hwr_df.to_csv('hwr_df.csv', index=False, encoding='utf-8')

In [57]:
!ls raw_data
!ls raw_data/variety

deadline  director_lists  filmcomment  hwreporter  variety
variety_columns_urls_records.csv  variety_reviews_urls_records.csv
variety_news_urls_records.csv


In [70]:
variety_columns_df = pd.read_csv('raw_data/variety/variety_columns_urls_records.csv')
variety_columns_df = variety_columns_df.drop('uid', axis=1)
print(variety_columns_df['category'].unique(), len(variety_columns_df))
variety_columns_df.drop_duplicates(keep='first',inplace=True) 
print(variety_columns_df['category'].unique(), len(variety_columns_df))

variety_reviews_df = pd.read_csv('raw_data/variety/variety_reviews_urls_records.csv')
variety_reviews_df = variety_reviews_df.drop('uid', axis=1)
print(variety_reviews_df['category'].unique(), len(variety_reviews_df))
variety_reviews_df.drop_duplicates(keep='first',inplace=True) 
print(variety_reviews_df['category'].unique(), len(variety_reviews_df))
variety_reviews_df.head()

variety_news_df = pd.read_csv('raw_data/variety/variety_news_urls_records.csv')
variety_news_df = variety_news_df.drop('uid', axis=1)
print(variety_news_df['category'].unique(), len(variety_news_df))
variety_news_df.drop_duplicates(keep='first',inplace=True) 
print(variety_news_df['category'].unique(), len(variety_news_df))
variety_news_df.head()

variety_df = pd.concat([variety_columns_df, variety_reviews_df, variety_news_df])
variety_df.insert(0, "media", "variety")
variety_df.head()

variety_df.to_csv('variety_df.csv', index=False, encoding='utf-8')

['columns'] 2535
['columns'] 2528
['reviews' 'category'] 42082
['reviews' 'category'] 41643
['news'] 145291
['news'] 143000


In [77]:
!ls raw_data/filmcomment

filmcomment_article_urls_records.csv  filmcomment_blog_urls_records.csv


In [84]:
filmcomment_article_df = pd.read_csv('raw_data/filmcomment/filmcomment_article_urls_records.csv')
filmcomment_article_df = filmcomment_article_df.drop('uid', axis=1)
print(filmcomment_article_df['category'].unique(), len(filmcomment_article_df))
filmcomment_article_df.drop_duplicates(keep='first',inplace=True)
filmcomment_article_df.head()

['article'] 1702


,category,url
0,article,https://www.filmcomment.com/article/jia-zhangk...
1,article,https://www.filmcomment.com/article/review-a-m...
2,article,https://www.filmcomment.com/article/directions...
3,article,https://www.filmcomment.com/article/review-the...
4,article,https://www.filmcomment.com/article/28th-annua...


In [85]:
filmcomment_blog_df = pd.read_csv('raw_data/filmcomment/filmcomment_blog_urls_records.csv')
filmcomment_blog_df = filmcomment_blog_df.drop('uid', axis=1)
print(filmcomment_blog_df['category'].unique(), len(filmcomment_article_df))
filmcomment_blog_df.drop_duplicates(keep='first',inplace=True)
filmcomment_blog_df.head()

['blog'] 1701


,category,url
0,blog,https://www.filmcomment.com/blog/interview-bre...
1,blog,https://www.filmcomment.com/blog/deep-focus-ma...
2,blog,https://www.filmcomment.com/blog/lois-weber-du...
3,blog,https://www.filmcomment.com/blog/making-case-c...
4,blog,https://www.filmcomment.com/blog/the-film-comm...


In [87]:
filmcomment_df = pd.concat([filmcomment_blog_df, filmcomment_article_df])
filmcomment_df.insert(0, "media", "filmcomment")
filmcomment_df.to_csv('filmcomment_df.csv', index=False, encoding='utf-8')

In [90]:
!ls raw_data/deadline

deadline_film_urls_records.csv


In [94]:
deadline_df = pd.read_csv('raw_data/deadline/deadline_film_urls_records.csv')
deadline_df = deadline_df.drop('uid', axis=1)
deadline_df.insert(0, "media", "deadline")
deadline_df.to_csv('deadline_df.csv', index=False, encoding='utf-8')

In [95]:
!ls

deadline_df.csv     hwr_df.csv		raw_data       variety_df.csv
filmcomment_df.csv  parse_scripts	sandbox.ipynb
gathering_scripts   processing_scripts	sandbox.py


In [96]:
df_1 = pd.read_csv('deadline_df.csv')
df_2 = pd.read_csv('hwr_df.csv')
df_3 = pd.read_csv('variety_df.csv')
df_4 = pd.read_csv('filmcomment_df.csv')

In [97]:
film_media_df = pd.concat([df_1, df_2, df_3, df_4])

In [105]:
film_media_df.loc[film_media_df.category == 'reviews', 'category'] = 'review'
film_media_df['category'].unique()

array(['film', 'news', 'review', 'columns', 'blog', 'article'],
      dtype=object)

In [106]:
len(film_media_df)

344225

In [108]:
film_media_df.head()

,media,category,url
0,deadline,film,https://deadline.com/2019/01/4-percent-challen...
1,deadline,film,https://deadline.com/2019/03/jared-harris-jare...
2,deadline,film,https://deadline.com/2019/03/sky-tin-star-tim-...
3,deadline,film,https://deadline.com/2019/02/jj-abrams-jedi-st...
4,deadline,film,https://deadline.com/2019/03/june-harding-dies...


In [107]:
film_media_df.to_csv('film_media_df.csv', index=False, encoding='utf-8')